In [1]:
import requests
url = "https://ressources.data.sncf.com/api/records/1.0/search/"

In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import *

engine = create_engine('sqlite:///db.sqlite')
Base = declarative_base()

Session = sessionmaker(bind=engine)
session = Session()

class Record(Base):
    __tablename__ = "Record"

    recordid = Column(String, nullable=True, primary_key=True)
    gare_code_uic = Column(String, nullable=True)
    type = Column(String, nullable=True)
    nature = Column(String, nullable=True)
    recordtype = Column(String, nullable=True)
    date = Column(String, nullable=True)

class Gare(Base):
    __tablename__ = "Gare"

    uic_code = Column(String, primary_key=True)
    departement_libellemin = Column(String)
    gare_regionsncf_libelle = Column(String)
    gare_alias_libelle_noncontraint = Column(String)
    adresse_cp = Column(String)
    code_gare = Column(String)
    lat = Column(REAL)
    long = Column(REAL)


Base.metadata.create_all(engine)


In [3]:
# APPEL API POUR TOUS LES RECORDS ENTRE 2016 ET 2021

for a in [2016, 2017, 2018, 2019, 2020, 2021]:
    for m in ["01","02","03","04","05","06","07","08","09","10","11","12"]:
        response = requests.get(f'{url}?dataset=objets-trouves-restitution&rows=-1&refine.date={str(a)}%2F{m}').json()
        try:
            for i in response['records']:
                try:
                    gare_code_uic = i['fields']['gc_obo_gare_origine_r_code_uic_c']
                except:
                    gare_code_uic = None
                    
                session.add(Record(recordid = i['recordid'],gare_code_uic = gare_code_uic,type = i['fields']['gc_obo_type_c'],nature = i['fields']['gc_obo_nature_c'],recordtype = i['fields']['gc_obo_nom_recordtype_sc_c'],date = i['fields']['date']))
            session.commit()
        except:
            pass


In [4]:
# APPEL API POUR TOUTES LES GARES

for g in requests.get(f'{url}?dataset=referentiel-gares-voyageurs&q=&rows=-1&sort=gare_alias_libelle_noncontraint&facet=departement_libellemin&facet=segmentdrg_libelle&facet=gare_agencegc_libelle&facet=gare_regionsncf_libelle&facet=gare_ug_libelle').json()['records']:
    
    try:
        gare_regionsncf_libelle = g['fields']['gare_regionsncf_libelle']
    except:
        gare_regionsncf_libelle = None
    
    try:
        departement_libellemin = g['fields']['departement_libellemin']
    except:
        departement_libellemin = None

    try:
        gare_alias_libelle_noncontraint = g['fields']['gare_alias_libelle_noncontraint']
    except:
        gare_alias_libelle_noncontraint = None

    try:
        adresse_cp = g['fields']['adresse_cp']
    except:
        adresse_cp = None

    try:
        code_gare = g['fields']['code_gare']
    except:
        code_gare = None

    try:
        lat = g['geometry']['coordinates'][1]
        long = g['geometry']['coordinates'][0]
    except:
        lat = None
        long = None
    
    try:
        session.add(
        Gare(
            uic_code = g['fields']['uic_code'],
            departement_libellemin = departement_libellemin,
            gare_regionsncf_libelle = gare_regionsncf_libelle,
            gare_alias_libelle_noncontraint = gare_alias_libelle_noncontraint,
            adresse_cp = adresse_cp,
            code_gare = g['fields']['code_gare'],
            lat = lat,
            long = long
            )
        )
        session.commit()
    except:
        pass
    

In [5]:
import pandas as pd

df = pd.read_sql_table('Record', 'sqlite:///db.sqlite')

In [6]:
df = df.dropna(axis=0)

In [7]:
df['date'] = df['date'].apply(lambda x: str(x)[:10])
df['date']

0         2016-01-06
1         2016-01-06
2         2016-01-06
3         2016-01-06
4         2016-01-06
             ...    
481297    2021-12-31
481298    2021-12-31
481299    2021-12-31
481300    2021-12-31
481301    2021-12-31
Name: date, Length: 481117, dtype: object